In [3]:
!pip install --upgrade gspread oauth2client gspread_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe
import pandas as pd


In [7]:
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
import pandas as pd
from gspread_dataframe import set_with_dataframe

# Authenticate and create the client
auth.authenticate_user()
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# Replace with the name of your Google Sheet
sheet_name = "FSBO Leads"  # or whatever name you gave it
worksheet = gc.open(sheet_name).sheet1

# Fetch existing data
data = worksheet.get_all_records()
df = pd.DataFrame(data)
df.head()


,Name,phone,Address,city,state,zip,price,url,Notes
0,John Doe,855-94-9800,xyz Test drive,Test,Test+,12345,120K,xyz.com,


In [9]:
# Create a test lead dictionary
test_lead = {
    "Name": "John Doe",
    "Phone": "123-456-7890",
    "Address": "123 Main St",
    "City": "Dallas",
    "State": "TX",
    "Zip": "75001",
    "Price": "$200,000",
    "URL": "https://example.com/fsbo123",
    "Notes": "Test lead added from Colab"
}

# Convert it to a DataFrame
new_lead_df = pd.DataFrame([test_lead])

# Get existing data
existing_data = worksheet.get_all_records()
existing_df = pd.DataFrame(existing_data)

# Append new lead to the existing data
updated_df = pd.concat([existing_df, new_lead_df], ignore_index=True)

# Write back to Google Sheet
worksheet.clear()  # Clear old data
set_with_dataframe(worksheet, updated_df)  # Write updated data


In [10]:
import requests
from bs4 import BeautifulSoup
import time
import random

# Replace with your target location
city = "dallas"
state = "tx"

# Zillow FSBO URL
url = f"https://www.zillow.com/{city}-{state}/fsbo/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Zillow loads data via JavaScript, so we can't fully scrape live listings easily.
# Instead, we'll simulate scraped data for now.

# Fake scraped lead (in real world this would come from parsing soup)
scraped_lead = {
    "Name": "Owner (FSBO)",
    "Phone": "",  # Zillow doesn't provide it without contact
    "Address": "456 Sample Ln",
    "City": city.title(),
    "State": state.upper(),
    "Zip": "00000",
    "Price": "$250,000",
    "URL": url,
    "Notes": "Example Zillow FSBO"
}

# Add to DataFrame
new_lead_df = pd.DataFrame([scraped_lead])

# Load current sheet
existing_data = worksheet.get_all_records()
existing_df = pd.DataFrame(existing_data)

# Check for duplicates (based on URL)
if not (existing_df["URL"] == scraped_lead["URL"]).any():
    updated_df = pd.concat([existing_df, new_lead_df], ignore_index=True)
    worksheet.clear()
    set_with_dataframe(worksheet, updated_df)
    print("✅ New Zillow lead added!")
else:
    print("⚠️ Lead already exists.")


✅ New Zillow lead added!


In [13]:
# Craigslist FSBO scraper (basic)
city = "dallas"
state = "tx"
craigslist_url = f"https://{city}.craigslist.org/d/real-estate-for-sale/search/rea"

# Send a request to the Craigslist page
response = requests.get(craigslist_url, headers=headers)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the listings (this is based on Craigslist page structure)
listings = soup.find_all('li', class_='result-row')

# Loop through each listing and extract information
for listing in listings:
    title = listing.find('a', class_='result-title').text
    link = listing.find('a', class_='result-title')['href']

    # Fake data extraction (You could expand this by scraping actual data from each listing page)
    scraped_lead = {
        "Name": "Owner (FSBO)",
        "Phone": "",
        "Address": title,
        "City": city.title(),
        "State": state.upper(),
        "Zip": "00000",  # Placeholder
        "Price": "$150,000",  # Placeholder
        "URL": link,
        "Notes": "Craigslist FSBO listing"
    }

    # Add to DataFrame
    new_lead_df = pd.DataFrame([scraped_lead])

    # Load current sheet
    existing_data = worksheet.get_all_records()
    existing_df = pd.DataFrame(existing_data)

    # Check for duplicates (based on URL)
    if not (existing_df["URL"] == scraped_lead["URL"]).any():
        updated_df = pd.concat([existing_df, new_lead_df], ignore_index=True)
        worksheet.clear()
        set_with_dataframe(worksheet, updated_df)
        print(f"✅ New Craigslist lead added: {title}")
    else:
        print(f"⚠️ Duplicate found: {title}")
